# Introduction
In This file, I will train a Yolov11 model to detect wall cracks and road cracks, The project will use the Yolov11 segmentation large model version that is previously trained on the COCO dataset. However, I will retrain it on the custom dataset to detect wall and road cracks.

# 1. Import libraries

In [1]:
from ultralytics import YOLO

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# 2. Importing the custom cracks dataset

- Dataset Name: Crack Computer Vision Dataset
- URL: https://universe.roboflow.com/apu-jimbr/crack-uuasm

- Dataset Description: The dataset has about 4000 images of cracks in roads, walls, and cement in general.
1. The Number of files in data\test\images is 322.
2. The Number of files in data\test\labels is 322.
3. The Number of files in data\train\images is 7072.
4. The Number of files in data\train\labels is 7072.
5. The Number of files in data\valid\images is 455.
6. The Number of files in data\valid\labels is 455.

In [3]:
def get_directory(directory, structure=['images', 'labels']):
    folders = os.listdir(directory)
    for folder in folders:
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            subfolders = os.listdir(folder_path)
            for final_folders in subfolders:
                if final_folders in structure:
                    final_folder_path = os.path.join(folder_path, final_folders)
                    files = [f for f in os.listdir(final_folder_path) if os.path.isfile(os.path.join(final_folder_path, f))]
                    num_files = len(files)
                    print(f"The Number of files in {final_folder_path} is {num_files}.")

get_directory("data")

The Number of files in data\test\images is 322.
The Number of files in data\test\labels is 322.
The Number of files in data\train\images is 14144.
The Number of files in data\train\labels is 7072.
The Number of files in data\valid\images is 910.
The Number of files in data\valid\labels is 455.


# 3. Training the model

In [4]:
model = YOLO('yolo11m-seg.pt')

In [5]:
# Train Settings
coco_path = "data/data.yaml"
epochs = 25
input_image_size = 640
patience = 3
batch = 32
val = True
cache = True
cache= 'disk'
plots = True
project = "Yolov11-test"
workers = 2

In [6]:
# Train the Model
results = model.train(data=coco_path, epochs=epochs, imgsz=input_image_size, patience=patience, batch=batch, val=val, cache=cache, plots=plots, project=project, workers=workers)

Ultralytics 8.3.228  Python-3.13.5 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 2080 SUPER, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=3, perspective=0.0, plots=True,

OutOfMemoryError: CUDA out of memory. Tried to allocate 400.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 12.72 GiB is allocated by PyTorch, and 87.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)